In [6]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from urllib.request import urlopen
import time
from selenium import webdriver

In [8]:
from math import ceil
from time import sleep
from random import randint
from bs4 import BeautifulSoup


from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd
import numpy as np
import requests
import json

In [2]:
chrome_path = r'D:\\Telecom 1e anne\\Telecom Period 1\\Jupyter\\MODS203\\Project\\chromedriver.exe'
driver = webdriver.Chrome(chrome_path)
# driver.get("https://www.kisskissbankbank.com/en/discover")

In [7]:
pip install webdriver_manager

You should consider upgrading via the 'd:\users\lenovo\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [3]:
base_url = 'https://www.kisskissbankbank.com'

In [9]:
url = f'{base_url}/en/discover?project[successful]=on&filter=all'

response = requests.get(url)
soup = BeautifulSoup(response.content, "html5lib")

script = json.loads(soup.find('script', class_='js-react-on-rails-component').text)
nb_projects = script['BROWSING']['projects']['totalCount']

nb_pages = ceil(nb_projects/9)

In [14]:
browser = webdriver.Chrome(ChromeDriverManager().install())
wait = WebDriverWait(browser, 100)


for page_number in range(1, nb_pages+1):
    url = f'{base_url}/en/discover?filter=all&project[successful]=on&page={page_number}'
    browser.get(url)
    sleep(2)
    
    for item in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.k-LegoGrid__item__content>a'))): 
        cats = item.find_element_by_css_selector('div.k-CrowdfundingCard__subtitle__container>p>span>span>span').text
        links.append({'url': item.get_attribute('href'), 'cats': cats})
    
    if page_number % 100 == 0:
        print(f'Page {page_number} / {nb_pages+1} treated')

[WDM] - Current google-chrome version is 86.0.4240
[WDM] - Get LATEST driver version for 86.0.4240
[WDM] - Driver [C:\Users\lenovo\.wdm\drivers\chromedriver\win32\86.0.4240.22\chromedriver.exe] found in cache


TimeoutException: Message: 


In [28]:
#function to get the html code from the requested url
def download_from_the_internet(url):
    try:
        return urlopen(url).read().decode('utf-8')
    except KeyboardInterrupt:
        raise
    except:
        return None

#function returns list of the tupels, where [0] – link to the project, [1] – category assigned to this project
def extract_links_from_html(html):  
    parser = BeautifulSoup(html)
#     parser = parser.find('div', attrs = {'class':'styles__StyledGrid-xsvioa-0 iBEewj kiss-BrowsingGrid'})
    parser = parser.find('div', attrs = {'class':'styles__StyledGrid-xsvioa-0'})
    return [(link.get('href'), link.find('div', attrs={'class':'k-CrowdfundingCard__subtitle__container'}).text) for link in parser.findAll('a')]

#functions return you list of the project links on the web-site
def get_all_projects_links(url, number):
    links = []
    for i in range(1,number):
        html = download_from_the_internet(url+'&page='+str(i))
#         print(html)
        links.extend(extract_links_from_html(html))
        time.sleep(5)
    return links

In [55]:
url = 'https://www.kisskissbankbank.com/en/discover?categories[music]=on&categories[adventure-and-sport]=on&categories[live-performance]=on&categories[film-and-video]=on&filter=all' #url with the filters applied
links = get_all_projects_links(url, 15)

In [30]:
url = 'https://www.kisskissbankbank.com/en/projects/effervescience-stressdefense'
html = requests.get(url)

In [17]:
soup = BeautifulSoup(html.text, 'html.parser')
comment = soup.findAll('li',attrs ={'data-test-id':'comments-tab'})
# comment
contri = soup.find_all('li',attrs ={'data-test-id':'backers-tab'})
# contri.get('href')
print(comment, contri)

[<li class="k-NavBar__listItem" data-test-id="comments-tab" id="comments-tab"><a class="k-NavBar__link" href="/en/projects/effervescience-stressdefense/tabs/comments">Comments</a></li>] [<li class="k-NavBar__listItem" data-test-id="backers-tab" id="backers-tab"><a class="k-NavBar__link" href="/en/projects/effervescience-stressdefense/tabs/backers">Backers</a></li>]


In [41]:
driver.get(url)

In [69]:
a = driver.find_element_by_id('owner_profile_button')
a.text
# re.sub("[^0-9]",'',a.text)

'Maison Bagatella Production'

In [35]:
# soup
html = driver.page_source
# html

In [18]:
comment = soup.find('span', attrs ={'class':'badge__StyledBadge-sc-7liuod-0 eucluz k-Badge k-Badge--spaced'})
print(comment)
intro = soup.find_all('div', attrs ={'class':'marger__StyledMarger-sc-1qqifp5-0 kFDJHh intro__StyledInfoBlock-sc-1wfo5yh-5 lncbNE'})
com = soup.find('span', attrs ={'role':'alert'})

None


In [59]:
#function to create a list of projects for future dataframe 
def extract_data_from_html(html, category, url):
    soup = BeautifulSoup(html)
    driver.get(url)
    title = soup.find('h1', attrs ={'data-test-id':'project-title'}).text
    infos = soup.find_all('div', attrs = {'class':'marger__StyledMarger-sc-1qqifp5-0 kFDJHh intro__StyledInfoBlock-sc-1wfo5yh-5 lncbNE'})
    '''
        Here we get the data about number of backers, amount of funded and target money.
        Also parsing percentage from the progress bar and calculating the 'is_successfull parameter'.
    '''
    for i in infos:
        if len(infos) == 2:
        #re.sub here is to clean the string of commas, symbols, etc.. Final output consitst only of digits. 
            funded = re.sub("[^0-9]", "",infos[1].find('div', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-bold info__StyledBigText-lcf1ol-0 clfvNf'}).text)
            target = re.sub("[^0-9]", "",infos[1].find('div', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-light info__StyledSmallText-lcf1ol-1 gDdVEG'}).text)
            backers = infos[0].find('div').text
        else:
            funded = re.sub("[^0-9]", "",infos[2].find('div', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-bold info__StyledBigText-lcf1ol-0 clfvNf'}).text)
            target = re.sub("[^0-9]", "",infos[2].find('div', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-weight-light info__StyledSmallText-lcf1ol-1 gDdVEG'}).text)
            backers = infos[0].find('div').text

        percentage_fund = soup.find('div', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-color-font1 k-u-line-height-normal k-u-weight-regular progress__StyledText-ycznm1-1 iXebEz'}).text.split('\xa0')[0]
        if int(percentage_fund) >= 100: is_successfull = True
        else: is_successfull = False

    '''
        Here parsing the rewards for the backers. Prices is a list of rewards prices. 
        Number of rewards is the lenght of the prices list.
    '''
    #re.sub the same to leave only digits
    prices = [int(re.sub("[^0-9]", "",i.text)) for i in soup.find('div', attrs= {'id':'rewards'}).find_all('h2', attrs={'class':'title__StyledTitle-sc-46lshq-0 iYeFEL k-RewardCard__title k-u-margin-bottom-double k-u-margin-top-none'})]
    min_price = min(prices)
    max_price = max(prices)
    number_rewards = len(prices)
    
    thumbnail_type = 'https://www.kisskissbankbank.com/' + soup.find('img', attrs = {'class':'k-ButtonImage__img'}).get('src')
    
    number_rewards = re.sub("[^0-9]","", driver.find_element_by_id('rewards-tab').text)
    num_news = re.sub("[^0-9]","", driver.find_element_by_id('news-tab').text)
    num_comments = re.sub("[^0-9]","", driver.find_element_by_id('comments-tab').text)
    '''
        Here parsing the owner web-page to get the number of the created projects.
    '''
    #going to the creator web-page and parsing the number of created projects
    html = download_from_the_internet('https://www.kisskissbankbank.com/'+soup.find('a', attrs = {'id':'owner_profile_button'}).get('href'))
    owner = BeautifulSoup(html)
    num_creater_projects = owner.find('span', attrs = {'class':'k-Badge k-HorizontalNav__badge'}).text.split('\n')[1]
    

    '''
        Here creating a dictionary with the obtained data and return in to the result list. 
        After all the data parsed, from this list of dictionaries a pandas DataFrame will be created.
    '''
    data = {'title':title, 
            'category':category,
            'is_successfull':is_successfull,
            'percentage_fund':percentage_fund,
            'funded':funded, 
            'target':target,  
            'backers':backers, 
            'thumbnail_type':thumbnail_type, 
            'number_rewards':number_rewards, 
            'min_price':min_price, 
            'max_price':max_price, 
            'num_news':num_news,
            'num_comments':num_comments, 
#             'num_contribution':num_contribution,
            'num_creater_projects':num_creater_projects}
      
    return data

In [6]:
#function to create a list of projects for future dataframe 
def extract_data_from_html(url, html):
    data = []
    driver.get(url)
    #Better to replace it also with selenium selector to obtain the page and go through the pagination
    soup = BeautifulSoup(html)
    content = soup.find('div', attrs = {'class':'styles__StyledGrid-xsvioa-0 iBEewj kiss-BrowsingGrid'})
    projects = content.find_all('a')
    for project in projects:
      #for every project creating a dictionary instance with the corresponding columns
      #parsing data from the card
        title = project.find('div', attrs={'class':'k-CrowdfundingCard__titleAndDesc'}).text
        category = project.find('div', attrs={'class':'k-CrowdfundingCard__subtitle__container'}).text
        infos = a.find('div', attrs={'class':'k-CrowdfundingCard__informations__infoContainer'})
        for i in infos:
            if len(infos) == 2:
                funded = infos[1].find('strong', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp'}).text
                target = infos[1].find('span', attrs={'class':'k-u-hidden@m-up'}).text
                backers = infos[0].find('strong', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp'}).text
            else:
                funded = infos[2].find('strong', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp'}).text
                target = infos[2].find('span', attrs={'class':'k-u-hidden@m-up'}).text
                backers = infos[0].find('strong', attrs={'class':'text__StyledText-sc-1jqe2sw-0 kKHxol k-u-weight-regular info__StyledText-h7ofxb-0 jfWZBp'}).text

        percentage_fund = project.find('div', attrs={'class':'k-CrowdfundingCard__progressBar'}).text.split()[0]
        if percentage_fund >= 100: is_successfull = True
        else: is_successfull = False
        #going to the project web-page
        #Here should be code to go to the project page
        tags = #?
        rewards = #?
        prices = #?
        min_price = #?
        max_price = #?
        in_2020 = #?
        thumbnail_type = #?
        number_rewards = re.sub("[^0-9]","", driver.find_element_by_id('rewards-tab').text)
        num_news = re.sub("[^0-9]","", driver.find_element_by_id('news-tab').text)
        num_comments = re.sub("[^0-9]","", driver.find_element_by_id('comments-tab').text)
        num_creater_projects = #?
        num_contributions = #?
        
        creator_name = driver.find_element_by_id('owner_profile_button').text
        
        data.append({'title':title, 
                   'category':category,
                   'is_successfull':is_successfull,
                   'percentage_fund':percentage_fund,
                   'funded':funded, 
                   'target':target, 
                   'in_2020':in_2020, 
                   'backers':backers, 
                   'thumbnail_type':thumbnail_type, 
                   'number_rewards':number_rewards, 
                   'min_price':min_price, 
                   'max_price':max_price, 
                   'num_news':num_news,
                   'num_comments':num_comments, 
                   'num_contributions':num_contributions,
                   'num_creater_projects':num_creater_projects})
      
    return data

SyntaxError: invalid syntax (<ipython-input-6-cf2b5944bdb4>, line 29)

In [61]:
result = []
for link in links[:5]:
    url, category = link[0], link[1]
    html = download_from_the_internet(url)
    result.append(extract_data_from_html(html, category, url))
    time.sleep(1)
    
df = pd.DataFrame(result)
df.to_csv('output.csv')

In [56]:
#recieve the html code of the web-page
html = download_from_the_internet('https://www.kisskissbankbank.com/en/projects/way-le-voyage-de-demain')
links[1]

('https://www.kisskissbankbank.com/en/projects/ob2', 'Music, Recording')

In [64]:
df = pd.DataFrame(result)
df.head()
# df.to_csv('output.csv')

,title,category,is_successfull,percentage_fund,funded,target,backers,thumbnail_type,number_rewards,min_price,max_price,num_news,num_comments,num_creater_projects
0,"TEASER Long-métrage ""L'APPÂT""","Feature length, Film & video, Digital art",True,152,3292,2160,58,https://www.kisskissbankbank.com/https://d3v4j...,,5,80,,9,1
1,ENREGISTREMENT DE L'ALBUM OB2 GREGORY OTT ET V...,"Music, Recording",False,98,9815,10000,186,https://www.kisskissbankbank.com//assets/avata...,,10,500,,45,1
2,Playschool l'album : LA GARDERIE,"Music, Film & video, Clips",True,117,3505,3000,92,https://www.kisskissbankbank.com//assets/avata...,,10,1000,4,22,1
3,Transat Jacques Vabre 2021 - Why it wouldn't b...,"Sport, Film & video, Tech",False,60,5985,10000,51,https://www.kisskissbankbank.com/https://d3v4j...,,10,1000,2,9,1
4,"""LA MANOEUVRE"" un film de Victor Azoulay","Film & video, Short film, Feature length",False,99,4970,5000,62,https://www.kisskissbankbank.com/https://d3v4j...,,10,1000,1,6,1
